## Getting the data
We obtain the data using Google Big Query.
In order to obtain the deposits and withdrawals transactions, just run the following querys:

### Withdrawals 
SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0x21a0adb6";

### Deposits
SELECT * FROM `bigquery-public-data.crypto_ethereum.transactions` WHERE `to_address` 
IN (SELECT `address` FROM `tornado_cash_transactions.tornadocontracts`) AND SUBSTR(`input`, 1, 10) = "0xb214faa5";

References here:
https://github.com/Phread420/tornado_bigquery/blob/main/Notes.md

This results are stored in the following files:
- withdraw_transactions.csv
- deposit_transactions.csv

After obtaining those two files, the next thing to do is to get the "recipient_addresses" that are making the withdrawl (if the user withdraws via a relayer, the relayer's account will be displayed in the "from_addres" field). That information is encoded in the "input" field of the withdrawal transactions.

The process to do so is in the following notebook:
https://github.com/lambdaclass/tornado_cash_anonymity_tool/blob/main/notebooks/complete_withdraw_data_set.ipynb

Note that the recipient addresses obtained in this df are in upper case and the addresses coming from the big query are in lower case, so you have to make the transformation when running the heuristic.

## Second heuristic - Preliminary implementation

### Description

If there is a deposit and a withdraw transaction with **unique** gas prices (e.g., 3.1415926 Gwei), then we consider the deposit and the withdraw transactions linked. The corresponding deposit transaction can be removed from any other withdraw transaction’s anonymity set.

In [1]:
# Import relevant packages
import pandas as pd
from tqdm import tqdm

In [5]:
# Load transactions data

withdraw_transactions_df = pd.read_csv("../data/lighter_complete_withdraw_txs.csv")
# Change recipient_address to lowercase.
withdraw_transactions_df["recipient_address"] = withdraw_transactions_df["recipient_address"].str.lower()
# Change block_timestamp field to be a timestamp object.
withdraw_transactions_df["block_timestamp"] = withdraw_transactions_df["block_timestamp"].apply(pd.Timestamp)

non_relayer_withdraw_transactions_df = withdraw_transactions_df[withdraw_transactions_df["from_address"]==withdraw_transactions_df["recipient_address"]]

deposit_transactions_df = pd.read_csv("../data/lighter_complete_deposit_txs.csv")
# Change block_timestamp field to be a timestamp object.
deposit_transactions_df["block_timestamp"] = deposit_transactions_df["block_timestamp"].apply(pd.Timestamp)

# tornado_addresses = pd.read_csv("../data/tornadocontracts_abi.csv", names=['address', 'contract_currency', 'value', '4'])

### Function summary: filter_by_unique_gas_price

Given a transactions DataFrame, it returns a dictionary with the unique gas prices as keys and a tuple containing the hash of that transaction and the timestamp. For example,

```
{111000302: ("0x387fcd87a", timestamp1)
```

In [7]:
def filter_by_unique_gas_price(transactions_df):
    
    # Count the appearances of each gas price in the transactions DataFrame.
    
    gas_prices_count = transactions_df["gas_price"].value_counts()
    
    # Filter the gas prices that are unique, i.e., the ones with a count equal to 1.
    # The gas prices are stored in unique_gas_prices
    
    unique_gas_prices = gas_prices_count[gas_prices_count == 1].keys()
    
    # A dataframe of the transactions that verify having these unique gas price values.
    
    temp_df = transactions_df[transactions_df["gas_price"].isin(unique_gas_prices)]
    
    # The DataFrame is transformed to a dictionary with only the relevant information.
    
    return {row.gas_price: (row.hash, row.block_timestamp) for row in temp_df.itertuples()}

### Function summary: filter_by_unique_gas_price_by_pool

Given a transactions DataFrame, it returns a dictionary with a tuple of the unique gas prices and tornado pool as keys and a tuple containing the hash of that transaction and the timestamp. For example, 

```
{(111000302, "0x24387248"): ("0x387fcd87a", timestamp1)

```

In [8]:
def filter_by_unique_gas_price_by_pool(transactions_df):
    
    # Count the appearances of each gas price by pool in the transactions DataFrame.
    
    gas_prices_count = deposit_transactions_df[["gas_price", "tornado_cash_address"]].value_counts()
    
    '''
    Filter the gas prices that are unique for each pool, i.e., the ones with a count equal to 1.
    The gas prices are stored in unique_gas_prices
    '''
    
    unique_gas_prices_by_pool = pd.DataFrame(gas_prices_count[gas_prices_count == 1])
    
    # Tuple set with the values (gas_price, tornado_cash_address) is made to filter efficiently
    
    tuple_set = set([(row.Index[0], row.Index[1]) for row in unique_gas_prices_by_pool.itertuples()])
    
    # A dataframe of the transactions that verify having these unique gas price values by pool.
    
    temp_df = pd.DataFrame(filter(lambda iter_tuple: (iter_tuple.gas_price, iter_tuple.tornado_cash_address) in tuple_set, deposit_transactions_df.itertuples()))
    
    # The DataFrame is transformed to a dictionary with only the relevant information.
    
    return {(row.gas_price, row.tornado_cash_address): (row.hash, row.block_timestamp) for row in temp_df.itertuples()}

A test of the function applied to our deposit transactions data.

### Function summary: same_gas_price_heuristic

This function receives a particular withdraw transaction and a dictionary with the unique gas price deposits, as returned by the function filter_by_unique_gas_price.

It returns a tuple:
* $(True, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction is found.
* $(False, None)$ when such a deposit is not found.

In [9]:
def same_gas_price_heuristic(withdrawal_transaction, unique_gas_price_deposit_dict):
    
    '''
    Check if the gas price of the given withdrawal transaction is within the dictionary keys and that
    the deposit was made earlier than the withdral.
    If it is, then a tuple is returned, (True, deposit_hash).
    If it is not, (False, None) is returned.
    '''
    
    if withdrawal_transaction.gas_price in set(unique_gas_price_deposit_dict.keys()) and (unique_gas_price_deposit_dict[withdrawal_transaction.gas_price][1] < withdrawal_transaction.block_timestamp):
        return (True, unique_gas_price_deposit_dict[withdrawal_transaction.gas_price][0])
    else:   
        return (False, None)

### Function summary: same_gas_price_heuristic_by_pool

This function receives a particular withdraw transaction and a dictionary with the unique gas price deposits made in each pool, just as returned by the function filter_by_unique_gas_price_by_pool.

It returns a tuple:
* $(True, deposit$ $hash)$ when a deposit transaction with the same gas price and belonging from the same pool (for example, 1ETH) as the withdrawal transaction is found.
* $(False, None)$ when such a deposit is not found.

In [10]:
def same_gas_price_heuristic_by_pool(withdrawal_transaction, unique_gas_price_deposit_dict):
    
    '''
    Check if the pair (gas_price, tornado_pool) of the given withdrawal transaction is within the dictionary keys and that
    the deposit was made earlier than the withdral.
    If it is, then a tuple is returned, (True, deposit_hash).
    If it is not, (False, None) is returned.
    '''
    
    if (withdrawal_transaction.gas_price, withdrawal_transaction.tornado_cash_address) in set(unique_gas_price_deposit_dict.keys()) and (unique_gas_price_deposit_dict[(withdrawal_transaction.gas_price, withdrawal_transaction.tornado_cash_address)][1] < withdrawal_transaction.block_timestamp):
        return (True, unique_gas_price_deposit_dict[(withdrawal_transaction.gas_price, withdrawal_transaction.tornado_cash_address)][0])
    else:   
        return (False, None)

### Function summary: apply_same_gas_price_heuristic

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [11]:
def apply_same_gas_price_heuristic(deposit_transactions_df, withdraw_transactions_df):
    
    # Get deposit transactions with unique gas prices.
    
    unique_gas_price_deposits_dict = filter_by_unique_gas_price(deposit_transactions_df)
    
    # Initialize an empty dictionary to store the linked transactions.
    
    withdrawal_to_deposit = {}
    
    # Iterate over the withdraw transactions.
    for withdraw_row in tqdm(withdraw_transactions_df.itertuples(), total=withdraw_transactions_df.shape[0], mininterval=10):     
        # Apply heuristic for the given withdraw transaction.
        same_gas_deposit_hash = same_gas_price_heuristic(withdraw_row, unique_gas_price_deposits_dict)
        
        # When a deposit transaction matching the withdraw transaction gas price is found, add
        # the linked transactions to the dictionary.
        if same_gas_deposit_hash[0]:
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[1]

    # Return the linked transactions dictionary.
    return withdrawal_to_deposit

### Function summary: apply_same_gas_price_heuristic_by_pool

Applies the heuristic to all the withdraw_transactions DataFrame, filtering also by the particular pool. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [12]:
def apply_same_gas_price_heuristic_by_pool(deposit_transactions_df, withdraw_transactions_df):
    
    # Get deposit transactions with unique gas prices.
    
    unique_gas_price_deposits_dict = filter_by_unique_gas_price_by_pool(deposit_transactions_df)
    
    # Initialize an empty dictionary to store the linked transactions.
    
    withdrawal_to_deposit = {}
    
    # Iterate over the withdraw transactions.
    for withdraw_row in tqdm(withdraw_transactions_df.itertuples(), total=withdraw_transactions_df.shape[0], mininterval=10):     
        # Apply heuristic for the given withdraw transaction.
        same_gas_deposit_hash = same_gas_price_heuristic_by_pool(withdraw_row, unique_gas_price_deposits_dict)
        
        # When a deposit transaction matching the withdraw transaction gas price is found, add
        # the linked transactions to the dictionary.
        if same_gas_deposit_hash[0]:
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[1]

    # Return the linked transactions dictionary.
    return withdrawal_to_deposit

We run the heuristic for the entire withdraw dataset

In [13]:
linked_transactions = apply_same_gas_price_heuristic(deposit_transactions_df, non_relayer_withdraw_transactions_df)

100%|██████████| 17037/17037 [00:07<00:00, 2326.89it/s]


In [14]:
linked_transactions = pd.DataFrame.from_dict({"withdraw_hashes": linked_transactions.keys(), "deposit_hashes": linked_transactions.values()})
linked_transactions

,withdraw_hashes,deposit_hashes
0,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
1,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
2,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
3,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
4,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
...,...,...
123,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
124,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
125,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...
126,(0xb07d06c8ca80ce4052954393d79b2f95d83034607d1...,(0xbcd359d0323dc74c7a246d98cae706c75a6b3b51820...


Now we run the heuristic that filters by pool, again applying it to the entire withdraw dataset.

In [15]:
linked_transactions_by_pool = apply_same_gas_price_heuristic_by_pool(deposit_transactions_df, non_relayer_withdraw_transactions_df)

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [12]:
linked_transactions_by_pool_df = pd.DataFrame.from_dict({"withdraw_hashes": linked_transactions_by_pool.keys(), "deposit_hashes": linked_transactions_by_pool.values()})
linked_transactions_by_pool_df

,withdraw_hashes,deposit_hashes
0,0xb0835cab1d39dfcc6e43f5382adb9a14eff1b6594066...,0x7734d2f258455fbf845ce3ddb889a711c0493a0fc93a...
1,0xb286bf92448d1551fd3be5fb97709d1015086118437d...,0xe33a7ab84bb4d31b487529e42f869b5f2310d9aae9c8...
2,0x3ef076bef26c1d3a8fd871ecbd125f98c2bb85d4fb4b...,0x4da8899d4764c3a8d64f95f1bf0c67b61dc00fed3381...
3,0xda1eb6593a724f270d6668932f874546a0e98317ff76...,0xd382985d27002cbd8c803b324a42dcc93e39955fb9a4...
4,0xb958620f035bca2c1776deeef03f642c3bfeb40b7da9...,0x52712f9e22e5b0c822d88ce78b16130d91064fd7f1ad...
...,...,...
1258,0x4611c3b8eb6b90d267295aa822878a4200e5257fa381...,0x1af19bf59b0a2c3877a784d7c0e9d879e0550236bb25...
1259,0x88e3e2d091efaa07f3ca7bae94d7cd6314ca68127d6b...,0x773d0041752549dbffa735b3a5778746789adcdce51b...
1260,0x2516a26c3ff0a0583c72b9b5666c36370789843ff85e...,0x658be4fc6bf021744611067c36319be83f49f011fa7a...
1261,0x9e1a17c0286f15358687ae7cb4f206ccfa14a08b5384...,0x1cdf20fc3fcf4ef61a3e3e45e579611e1070a0de8771...
